# [admin] Users command


The `users` command in `admin` scope could help you manage users.


## Setup PrimeHub Python SDK


In [1]:
from primehub import PrimeHub, PrimeHubConfig
ph = PrimeHub(PrimeHubConfig())

if ph.is_ready():
    print("PrimeHub Python SDK setup successfully")
else:
    print("PrimeHub Python SDK couldn't get the group information, follow the 00-getting-started.ipynb to complete it")

PrimeHub Python SDK setup successfully


## Help documentation

In [2]:
help(ph.admin.users)

Help on AdminUsers in module primehub.admin_users object:

class AdminUsers(primehub.Helpful, primehub.Module)
 |  AdminUsers(primehub: primehub.PrimeHub, **kwargs)
 |  
 |  Method resolution order:
 |      AdminUsers
 |      primehub.Helpful
 |      primehub.Module
 |      primehub.HTTPSupport
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  add_group(self, id: str, group_id)
 |      Add group connection to a user by id
 |      
 |      :type id: str
 |      :param id: the id of a user
 |      :type group_id: str
 |      :param group_id: group id
 |      
 |      :rtype dict
 |      :return a user with groups only
 |  
 |  create(self, config)
 |      Create a user
 |      
 |      :type config: dict
 |      :param config: The configurations for creating a user
 |      
 |      :rtype dict
 |      :return The user
 |  
 |  delete(self, id: str) -> dict
 |      Delete an user by id
 |      
 |      :type id: str
 |      :param id: the id of the user
 |      
 |      :rty

## User management

In [3]:
!primehub admin users help

Usage: 
  primehub admin users <command>

Manage users

Available Commands:
  add-group            Add group connection to a user by id
  create               Create a user
  delete               Delete an user by id
  get                  Get an user by id
  list                 List users
  list-groups          List groups of a user by id
  remove-group         Remove group connection from a user by id
  reset-password       Reset password by id
  update               Update the user

Options:
  -h, --help           Show the help

Global Options:
  --config CONFIG      Change the path of the config file (Default: ~/.primehub/config.json)
  --endpoint ENDPOINT  Override the GraphQL API endpoint
  --token TOKEN        Override the API Token
  --group GROUP        Override the current group
  --json               Output the json format (output human-friendly format by default)


For `create` and `update` actions are needed a configuration to mutate a user. Here is the fields table:


### Fields

| field | required | type | description |
| --- | --- | --- | --- |
| username | required | string | lower case alphanumeric characters, '-', '.', and underscores ("_") are allowed, and must start with a letter or numeric.` |
| email | optional | string | a valid email |
| firstName | optional | string | |
| lastName | optional | string | |
| isAdmin | optional | boolean | grant the administrator role to the user |
| volumeCapacity | optional | int | customize the size of the user volume. unit: `GB`|
| groups | optional | assign the user to groups | please see the `connect` examples |

These fields are only used with email activation (only for `create`):

| field | required | type | description |
| --- | --- | --- | --- |
| sendEmail | optional | boolean | send an activation email to the user. (it worked if the smtp was set)|
| resetActions.set | optional | string[] | ask for actions, valid actions: `['VERIFY_EMAIL', 'UPDATE_PASSWORD']` |
| expiresIn | optional | int | expired duration for the activation email |


## Examples

You could find [more examples on our github](https://github.com/InfuseAI/primehub-python-sdk/blob/main/docs/CLI/admin/users.md).

In [4]:
import pandas as pd

#### CRUD: User

In [5]:
# Create a user with admin role
config = {
  "username": "user-admin-from-jupyter",
  "groups": {
    "connect": [
      {
        "id": ph.group_id
      }
    ]
  },
  "isAdmin": True
}
data = ph.admin.users.create(config)

In [6]:
# List users
user_df = pd.DataFrame(ph.admin.users.list())
user_df[user_df['username'] == config['username']]

,id,username,email,firstName,lastName,enabled,isAdmin
20,01711a38-da60-4840-91e0-68094105502f,user-admin-from-jupyter,None,None,None,True,True


In [7]:
# Get the user details
ph.admin.users.get(data['id'])

{'id': '01711a38-da60-4840-91e0-68094105502f',
 'username': 'user-admin-from-jupyter',
 'email': None,
 'firstName': None,
 'lastName': None,
 'enabled': True,
 'isAdmin': True,
 'volumeCapacity': None,
 'groups': [{'id': '0449b887-5c40-44ba-a945-41bc21c8d15a',
   'name': 'InfuseAI',
   'displayName': 'InfuseAI',
   'quotaCpu': 6,
   'quotaGpu': 2}]}

In [8]:
# Update the user config setting.
update_config = {"isAdmin": False}
data = ph.admin.users.update(data['id'], update_config)
ph.admin.users.get(data['id'])

{'id': '01711a38-da60-4840-91e0-68094105502f',
 'username': 'user-admin-from-jupyter',
 'email': None,
 'firstName': None,
 'lastName': None,
 'enabled': True,
 'isAdmin': False,
 'volumeCapacity': None,
 'groups': [{'id': '0449b887-5c40-44ba-a945-41bc21c8d15a',
   'name': 'InfuseAI',
   'displayName': 'InfuseAI',
   'quotaCpu': 6,
   'quotaGpu': 2}]}

In [9]:
# reset_password
ph.admin.users.reset_password(id=data['id'], password="test_reset_password", temporary=True)

{'id': '01711a38-da60-4840-91e0-68094105502f'}

In [10]:
# Delete the user
ph.admin.users.delete(data['id'])

{'id': '01711a38-da60-4840-91e0-68094105502f',
 'username': 'user-admin-from-jupyter'}

#### CRUD: User <-> Group

In [11]:
# Create a user with admin role
config = {
  "username": "user-admin-from-jupyter-group",
  "groups": {
    "connect": [
      {
        "id": ph.group_id
      }
    ]
  },
  "isAdmin": True
}
data = ph.admin.users.create(config)

In [12]:
# Show the user group list

user_group_df = pd.DataFrame(ph.admin.users.list_groups(data["id"]))
user_group_df

,id,name,displayName
0,0449b887-5c40-44ba-a945-41bc21c8d15a,InfuseAI,InfuseAI


In [13]:
# Add the user into group
groups_df = pd.DataFrame(ph.admin.groups.list())
groups_id = groups_df["id"][0]
ph.admin.users.add_group(data["id"], groups_id)

In [14]:
# Show the user group list

user_group_df = pd.DataFrame(ph.admin.users.list_groups(data["id"]))
user_group_df

,id,name,displayName
0,71ac1e32-65fa-4e8e-a735-ba282e3149b1,AnotherTestGroup,Another Test Group
1,0449b887-5c40-44ba-a945-41bc21c8d15a,InfuseAI,InfuseAI


In [15]:
# Remove the user out of group
ph.admin.users.remove_group(data["id"], groups_id)

In [16]:
# Show the user group list

user_group_df = pd.DataFrame(ph.admin.users.list_groups(data["id"]))
user_group_df

,id,name,displayName
0,0449b887-5c40-44ba-a945-41bc21c8d15a,InfuseAI,InfuseAI


In [17]:
# Delete the user
ph.admin.users.delete(data['id'])

{'id': '29541eb1-42cd-4b94-ac93-04d798986885',
 'username': 'user-admin-from-jupyter-group'}